In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [11]:
import cv2
import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
import time
import math
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.utils.data import Dataset
from tqdm import tqdm
from sklearn import metrics
import torch.nn.functional as F
import random
import sys
import shutil
import albumentations
from albumentations import pytorch as AT
from pytorchcv.model_provider import get_model as ptcv_get_model


from efficientnet_pytorch import EfficientNet

In [3]:
import scipy.special
sigmoid = lambda x: scipy.special.expit(x)

SEED = 323
base_dir =  '../plant-pathology-2020-fgvc7'

def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYHTONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(SEED)

In [4]:
sub = pd.read_csv( base_dir + '/sample_submission.csv')

In [5]:
sub.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.25,0.25,0.25,0.25
1,Test_1,0.25,0.25,0.25,0.25
2,Test_2,0.25,0.25,0.25,0.25
3,Test_3,0.25,0.25,0.25,0.25
4,Test_4,0.25,0.25,0.25,0.25


In [6]:
class PlantDataset(Dataset):
    
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):

        row = self.df.iloc[idx]
        path = base_dir + '/images/' + row.image_id + '.jpg'
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=image)
            
        image = image['image']
            
        return image

In [7]:
def pre_trans(x, cols, rows):
    return (x * 2.0 - 1.0)

# test_transform = albumentations.Compose([
#     albumentations.SmallestMaxSize(IMG_SIZE),
#     albumentations.CenterCrop(IMG_SIZE, IMG_SIZE),
#     albumentations.Lambda(image = pre_trans),
#     AT.ToTensor(),
#     ])

In [8]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x):
        return torch.cat([self.mp(x), self.ap(x)], 1)
    
def mish(input):
    return input * torch.tanh(F.softplus(input))
       
class Mish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input):
        return mish(input)

In [9]:
TTA = 16
BATCH_SIZE = 64
all_pred = []

In [14]:
################
EXP = 9
FOLD = 5
IMG_SIZE = 384
################

tta_transform = albumentations.Compose([
    albumentations.Resize(IMG_SIZE, IMG_SIZE),
    albumentations.RandomRotate90(p=0.5),
    albumentations.Transpose(p=0.5),
    albumentations.Flip(p=0.5),
    albumentations.Lambda(image = pre_trans),
    AT.ToTensor(),
    ])

testset       = PlantDataset(sub, transform = tta_transform)
test_loader   = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

pred_list = []
for fold in range(FOLD):
    print('*********Fold%d************'%fold)
    ############################################
#     EfficientNet.from_pretrained("efficientnet-b5", advprop=True)
    model_conv = ptcv_get_model("seresnext101_32x4d", pretrained=True)
#     model_conv._dropout = nn.Dropout(p=0.5)
#     model_conv._avg_pooling = AdaptiveConcatPool2d(1)
    model_conv.features.final_pool = nn.AdaptiveAvgPool2d(1)
#     model_conv._fc = nn.Sequential(nn.Linear(2048*2,256), Mish(), nn.Dropout(p=0.5), nn.Linear(256,4))
    model_conv.output = nn.Sequential(nn.Linear(2048,256), Mish(), nn.Dropout(p=0.5), nn.Linear(256,4))
    model_conv.cuda()
#     model_conv = EfficientNet.from_name('efficientnet-b5')
#     model_conv._dropout = nn.Dropout(p=0.5)
#     model_conv._avg_pooling = AdaptiveConcatPool2d()
#     model_conv._fc = nn.Sequential(nn.Linear(2048*2,256), Mish(), nn.Dropout(p=0.5), nn.Linear(256,4))

    model_conv.load_state_dict(torch.load('./exp/exp' + str(EXP) + '/efficientnet-b5-best' + str(fold) + '.pth'))
    
#     model_conv.cuda()
    model_conv.eval()
    ############################################
    for tta in range(TTA):
        y_pred_val = np.zeros((len(sub), 4))
        with torch.no_grad():
            for idx, imgs in enumerate(test_loader):
                imgs = imgs.cuda()
                output_test = model_conv(imgs) 
                b = output_test.detach().cpu().numpy()
                y_pred_val[idx*BATCH_SIZE:idx*BATCH_SIZE+b.shape[0]] = b

        pred_list.append(y_pred_val)

pred = sum(pred_list)/len(pred_list)
all_pred.append(pred)

*********Fold0************
*********Fold1************
*********Fold2************
*********Fold3************
*********Fold4************


In [15]:
pred = all_pred[0] * 1
print(pred)

[[-0.90366999 -0.34871876  2.66702873 -1.08475423]
 [-0.84850629 -0.50506409  2.67554153 -0.97004228]
 [-0.80470105 -0.96084007 -0.82845077  2.75989918]
 ...
 [-0.85354409 -0.28752739  2.654789   -1.16837686]
 [ 2.76936722 -0.65623806 -0.89259077 -0.81934151]
 [-0.79226592 -0.82261989 -0.96014618  2.73680891]]


In [16]:
sub.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.25,0.25,0.25,0.25
1,Test_1,0.25,0.25,0.25,0.25
2,Test_2,0.25,0.25,0.25,0.25
3,Test_3,0.25,0.25,0.25,0.25
4,Test_4,0.25,0.25,0.25,0.25


In [17]:
def softmax(X, theta = 1.0, axis = None):

    # make X at least 2d
    y = np.atleast_2d(X)

    # find axis
    if axis is None:
        axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)

    # multiply y against the theta parameter,
    y = y * float(theta)

    # subtract the max for numerical stability
    y = y - np.expand_dims(np.max(y, axis = axis), axis)

    # exponentiate y
    y = np.exp(y)

    # take the sum along the specified axis
    ax_sum = np.expand_dims(np.sum(y, axis = axis), axis)

    # finally: divide elementwise
    p = y / ax_sum

    # flatten if X was 1D
    if len(X.shape) == 1: p = p.flatten()

    return p

In [18]:
pred = softmax(pred, axis=1)
print(pred)

[[0.02556392 0.04452866 0.90857779 0.02132964]
 [0.02686958 0.03788052 0.9114553  0.0237946 ]
 [0.02620731 0.02241878 0.02559222 0.92578169]
 ...
 [0.02711224 0.0477511  0.90534714 0.01978952]
 [0.92094112 0.02995783 0.02365177 0.02544928]
 [0.02709442 0.02628435 0.02290711 0.92371411]]


In [20]:
sub['healthy'] = pred[:,0]
sub['multiple_diseases'] = pred[:,1]
sub['rust'] = pred[:,2]
sub['scab'] = pred[:,3]
sub.to_csv('submission.csv', index=False)
sub.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.025564,0.044529,0.908578,0.021330
1,Test_1,0.026870,0.037881,0.911455,0.023795
2,Test_2,0.026207,0.022419,0.025592,0.925782
3,Test_3,0.948390,0.009319,0.021505,0.020786
4,Test_4,0.026110,0.023385,0.929230,0.021275
